In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
#from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU, Bidirectional
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from scipy.io import arff
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, TimeDistributed
import h5py
from sklearn.model_selection import KFold,cross_val_score
kfold=KFold(5,True,3)
cvscores = []

In [ ]:
#load training dataset
# Name for the column  or Features Map
columns_to_named = ["frameNumber", "frameTime", "frameLen", "ethSrc", "ethDst",
       "ipSrc", "ipDst", "ipProto", "ipLen", "tcpLen", "tcpSrcport",
       "tcpDstport", "Value", "normality"]
    
    # Read the Dataset and Rename the Column
df = pd.read_csv("/content/drive/My Drive/Preprocessed_data.csv", header=0,names=columns_to_named)

col_norm = ['frameNumber', 'frameTime', 'frameLen', 'ethSrc', 'ethDst',
       'ipSrc', 'ipDst', 'ipProto', 'ipLen', 'tcpLen', 'tcpSrcport',
       'tcpDstport', 'Value']
    
    # Normalization using Custom Lambda Function or Min-max 
    
#df1_norm = df[col_norm].apply(lambda x :( (x - x.min()) / (x.max()-x.min()) ) )
X = df[col_norm]
#X = preprocessing.normalize(X)
Y = df['normality']

from sklearn.preprocessing import label_binarize
y = label_binarize(Y, classes=[0, 1, 2, 3, 4, 5])
n_classes = y.shape[1]
print(n_classes)

6


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

for train, test in kfold.split(X,Y):
    X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

    x_train = np.array(X_train)
    x_test = np.array(X_test)
    y_train1 = np.array(y_train)
    y_test1 = np.array(y_test)

    y_train= to_categorical(y_train1)
    y_test= to_categorical(y_test1)

    model = Sequential()
    model.add(Bidirectional(LSTM(80, input_dim=13, return_sequences=True)))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(40)))
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(6, activation='softmax'))

    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy','mae'])
    model.fit(X_train, y_train, epochs=15, batch_size=64, verbose=1)
    scores = model.evaluate(X_test, y_test, verbose=1)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Epoch 1/15
5968/5968 [==============================] - 42s 7ms/step - loss: 0.0410 - accuracy: 0.9903 - mae: 0.0066
Epoch 2/15
5968/5968 [==============================] - 38s 6ms/step - loss: 0.0130 - accuracy: 0.9971 - mae: 0.0017
Epoch 3/15
5968/5968 [==============================] - 37s 6ms/step - loss: 0.0094 - accuracy: 0.9975 - mae: 0.0014
Epoch 4/15
5968/5968 [==============================] - 37s 6ms/step - loss: 0.0077 - accuracy: 0.9978 - mae: 0.0012
Epoch 5/15
5968/5968 [==============================] - 38s 6ms/step - loss: 0.0061 - accuracy: 0.9982 - mae: 9.6724e-04
Epoch 6/15
5968/5968 [==============================] - 37s 6ms/step - loss: 0.0054 - accuracy: 0.9984 - mae: 8.5835e-04
Epoch 7/15
5968/5968 [==============================] - 37s 6ms/step - loss: 0.0043 - accuracy: 0.9986 - mae: 7.0389e-04
Epoch 8/15
5968/5968 [==============================] - 37s 6ms/step - loss: 0.0041 - accuracy: 0.9988 - mae: 6.3619e-04
Epoch 9/15
5968/5968 [==========================

In [ ]:
target_names = ['Normal', 'WrongSetup', 'DDoS', 'Probing', 'Scan', 'MITM']

In [ ]:
### train_accuracy
import sklearn.metrics as metrics
expected = y_train
predicted = model.predict(X_train)
data = classification_report(expected,np.round(predicted), target_names=target_names)
print(data)

print("Accuracy_Train:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision_Train:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall_Train:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score_Train:",metrics.f1_score(expected, np.round(predicted), average='weighted'))
print("ROC_AUC_Train:",metrics.roc_auc_score(expected, predicted,multi_class="ovr",average='weighted'))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00     63266
  WrongSetup       1.00      1.00      1.00     65753
        DDoS       1.00      1.00      1.00     63212
     Probing       1.00      1.00      1.00     63179
        Scan       1.00      1.00      1.00     63278
        MITM       1.00      1.00      1.00     63252

   micro avg       1.00      1.00      1.00    381940
   macro avg       1.00      1.00      1.00    381940
weighted avg       1.00      1.00      1.00    381940
 samples avg       1.00      1.00      1.00    381940

Accuracy_Train: 0.9996858145258417
Precision_Train: 0.9997985311298035
Recall_Train: 0.9996858145258417
f1-score_Train: 0.9997419903462866
ROC_AUC_Train: 0.9999998849726602


In [ ]:
### test_accuracy
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error,roc_auc_score,cohen_kappa_score)
import sklearn.metrics as metrics
expected = y_test
predicted = model.predict(X_test)
data = classification_report(expected,np.round(predicted), target_names=target_names)
print(data)
)
print("Accuracy:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score:",metrics.f1_score(expected, np.round(predicted), average='weighted'))
print("ROC_AUC:",metrics.roc_auc_score(expected, predicted,multi_class="ovr",average='weighted'))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00     15769
  WrongSetup       1.00      1.00      1.00     16532
        DDoS       1.00      1.00      1.00     15808
     Probing       1.00      1.00      1.00     15823
        Scan       1.00      1.00      1.00     15774
        MITM       1.00      1.00      1.00     15780

   micro avg       1.00      1.00      1.00     95486
   macro avg       1.00      1.00      1.00     95486
weighted avg       1.00      1.00      1.00     95486
 samples avg       1.00      1.00      1.00     95486

Accuracy: 0.9997172360346019
Precision: 0.9997802289461473
Recall: 0.9997172360346019
f1-score: 0.999748572244343
ROC_AUC: 0.9999999600469864
